In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# import sys
# !{sys.executable} -m pip install optuna

In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [4]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
pd.set_option("display.max_rows", 500)

In [6]:
csv_file_folder =  '../../data/input'

file_folder =  '../../data/feature'

In [8]:
mytrial = []

In [18]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [19]:
columns

Index(['atom_index_0', 'atom_index_1', 'type', 'NH', 'inv_dist1E', 'linkM1',
       'ID', 'adN2', 'yukawa_H.y', 'coulomb_H.x',
       ...
       'atom_index_1_eigv_gap', 'atom_index_1_fiedler_eig',
       'atom_index_1_connectedness', 'atom_index_1_sv_min',
       'atom_index_1_coulomb_mean', 'atom_index_1_sv_0', 'atom_index_1_sv_1',
       'atom_index_1_sv_2', 'atom_index_1_sv_3', 'atom_index_1_sv_4'],
      dtype='object', length=770)

In [9]:
LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.2,
    'num_leaves': 256,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0
}

In [13]:
model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=100, early_stopping_rounds=200)

y_pred = model.predict(X_val)
np.log(mean_absolute_error(y_val, y_pred))

Training until validation scores don't improve for 200 rounds.
[100]	training's l1: 0.843619	valid_1's l1: 0.941455
[200]	training's l1: 0.685367	valid_1's l1: 0.832333
[300]	training's l1: 0.594306	valid_1's l1: 0.782328
[400]	training's l1: 0.530607	valid_1's l1: 0.751125
[500]	training's l1: 0.478117	valid_1's l1: 0.728364
[600]	training's l1: 0.437059	valid_1's l1: 0.712559
[700]	training's l1: 0.40351	valid_1's l1: 0.701606
[800]	training's l1: 0.372437	valid_1's l1: 0.691977
[900]	training's l1: 0.345029	valid_1's l1: 0.684375
[1000]	training's l1: 0.320128	valid_1's l1: 0.6775
[1100]	training's l1: 0.297154	valid_1's l1: 0.672255
[1200]	training's l1: 0.277357	valid_1's l1: 0.667895
[1300]	training's l1: 0.258877	valid_1's l1: 0.663758
[1400]	training's l1: 0.242174	valid_1's l1: 0.660598
[1500]	training's l1: 0.227386	valid_1's l1: 0.657729
Did not meet early stopping. Best iteration is:
[1500]	training's l1: 0.227386	valid_1's l1: 0.657729


-0.4189623189116563

In [50]:
perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

In [61]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', [col]+columns_comb, 'error', error, 'error_train', error_train)
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if len(columns_comb)>70:
        break
    

col ['tertiary_angle_0'] error 1.6276652945323087 error_train 1.6261026767490179
col ['atom_1_bond_lengths_mean', 'tertiary_angle_0'] error 1.1251459463353315 error_train 1.1111938950459355
col ['inv_dist1R', 'tertiary_angle_0', 'atom_1_bond_lengths_mean'] error 0.9434274711113362 error_train 0.8938767670030703
col ['yukawa_H.y', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R'] error 0.8742410993931745 error_train 0.8004909545883621
col ['d_4_3', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y'] error 0.7738920824296573 error_train 0.6830555912733778
col ['mulliken_atom_0', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3'] error 0.5050918243835566 error_train 0.26942102956554553
col ['dist_O_0_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0'] error 0.2989170075313587 error_train 0.04023621532048349
col ['dist_to_type_0_mean', 'tertiary_angle_0', 'atom_1_bo

col ['dist_C_1_y', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div'] error -0.4160091924309197 error_train -0.9318140304801948
col ['dist_N_0_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist

col ['dist_to_type_std', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1'] error -0.4409291868459885 error_train -0.974547325673199
col ['tertiary_angle_5', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken

col ['distC1', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0'] error -0.48240848234212685 err

col ['d_5_0', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_an

...

col ['tertiary_distance_8', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0'

col ['gap', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_angl

col ['mulliken_max', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tert

col ['adH1', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_ang

col ['atom_index_1_sv_min', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0'

In [66]:
X_data = df_train2[columns_comb].values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print('col', columns_comb, 'error', error, 'error_train', error_train)

col ['tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_angle_6', '

In [ ]:
-0.4239748046183735 -1.4237774481571122 10
-0.43376108984538503 -1.4402760557422896 8
-0.40170720756899114 -1.4836995674002746 5
-0.368376392396792 -1.5932999128808094 3

In [88]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

group_kfold = GroupKFold(n_splits=8)
splits = group_kfold.split(X_data, y_data, df_train2.group)
for train_index, test_index in splits:
    break
X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print(error, error_train)

perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

-0.4324559369556779 -1.436310972165935


In [89]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
    
    if len(columns_comb)>70:
        break
    

col tertiary_angle_0 0 error 1.6222798164456236 error_train 1.6293130461726788
col inv_dist1R 1 error 1.3769224028122156 error_train 1.3682046457081336
col d_4_3 2 error 1.0526559798074329 error_train 0.9622265150615955
col yukawa_H.y 3 error 0.8880966852477664 error_train 0.7545132450913669
col mulliken_atom_0 4 error 0.6699714302742804 error_train 0.4381590087846014
col dist_to_type_mean 5 error 0.5723210796938699 error_train 0.31897853564242257
col dist_O_0_x 6 error 0.35635604998565557 error_train 0.07690433869670563
col atom_1_n_bonds 7 error 0.3479806235557046 error_train 0.06708991647819564
col dist_to_type_1_mean 8 error 0.34027981596648077 error_train 0.057885699625021256
col atom_1_bond_lengths_mean 9 error 0.20992680506741004 error_train -0.061747579339364166
col dist_xyz 10 error 0.20773890811152795 error_train -0.05768874175371354
col dist_C_0_y 11 error 0.15185049041648496 error_train -0.14306861964440287
col dist_to_type_std 12 error 0.15375433387730386 error_train -0.14

col molecule_atom_index_1_dist_max_diff 67 error -0.5038497510449675 error_train -1.1052230933637575
col tertiary_angle_12 67 error -0.5006790960752652 error_train -1.1225317411661033
col atom_index_1_dist_q.99 67 error -0.5019650905250351 error_train -1.1215623925888072
col atom_index_1_fiedler_eig 67 error -0.5004717035466232 error_train -1.1126256231821572
col adH2 67 error -0.5020626440704913 error_train -1.0996522236829769
col mulliken_min 67 error -0.4985403598207949 error_train -1.1120304697935761
col tertiary_atom_8 67 error -0.5035643875969926 error_train -1.1096691166947008
col atom_index_0_fiedler_eig 67 error -0.5001937769057281 error_train -1.1186863239693425
col tertiary_distance_13 67 error -0.4978514000970518 error_train -1.100724171957628
col yukawa_O.x 67 error -0.503149493115294 error_train -1.1022745443558095
col molecule_name.1 67 error -0.5024691485382106 error_train -1.1141206786580724
col vander_O.x 67 error -0.497348869017027 error_train -1.1035197890009252
col

col dist_xy 68 error -0.5004906837105771 error_train -1.1108043992844692
col tertiary_atom_12 68 error -0.5005180560132533 error_train -1.1080521685796514
col atom_index_0_p_lap_core 68 error -0.49967140502891894 error_train -1.1132839033101694
col tertiary_atom_13 68 error -0.5053244666211033 error_train -1.1142339956235725
col dist_to_type_mean_yz 68 error -0.502770958637726 error_train -1.1138566856407415
col dist_mean_bond_y 68 error -0.5067659085663165 error_train -1.104472462214877
col tertiary_angle_19 68 error -0.5009783479764099 error_train -1.1060569058859087
col c_z 68 error -0.5031241947803698 error_train -1.1146866010878864
col dist_x 68 error -0.5014339886041407 error_train -1.111321903085337
col z_dist 68 error -0.5020520676432993 error_train -1.1158614504744202
col atom_index_0_t_dist_factors_0 68 error -0.5017064576379667 error_train -1.1104905908219254
col tertiary_angle_20 68 error -0.5043191194268607 error_train -1.1006907901293295
col atom_index_1_p_dist_factors 68

In [90]:
columns_comb

['tertiary_angle_0',
 'inv_dist1R',
 'd_4_3',
 'yukawa_H.y',
 'mulliken_atom_0',
 'dist_to_type_mean',
 'dist_O_0_x',
 'atom_1_n_bonds',
 'dist_to_type_1_mean',
 'atom_1_bond_lengths_mean',
 'dist_xyz',
 'dist_C_0_y',
 'd_3_2',
 'atom_index_1_ hybridization',
 'atom_index_1_cycle_size_mean',
 'dist_O_0_y',
 'eem_1',
 'inv_distPE',
 'd_4_2',
 'inv_distPR',
 'dist_no_bond_min_y',
 'dist_H_2_x',
 'dist_H_1_x',
 'tertiary_distance_2',
 'dist_C_3_x',
 'dist_O_1_x',
 'atom_1_bond_lengths_std',
 'dist_C_2_y',
 'dist_C_2_x',
 'mulliken_atom_1',
 'cos_center1',
 'dist_O_1_y',
 'tertiary_angle_3',
 'dist_H_2_y',
 'dist_N_0_y',
 'dist_C_1_y',
 'inv_dist1E',
 'distance_y',
 'tertiary_angle_2',
 'dist_N_0_x',
 'd_2_1',
 'molecule_atom_index_0_dist_max_div',
 'adC1',
 'adN1',
 'd_4_0',
 'dist_C_3_y',
 'atom_3',
 'distC0',
 'tertiary_distance_4',
 'tertiary_angle_5',
 'd_5_1',
 'molecule_atom_index_1_dist_min_diff',
 'dist_C_4_y',
 'dist_H_0_y',
 'cos_f0',
 'd_5_0',
 'tertiary_distance_3',
 'd_5_2',


In [91]:
df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

df_train2 = df_train2[df_train2['type']==1]
df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
df_train2 = df_train2.fillna(0)

In [94]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

group_kfold = GroupKFold(n_splits=8)
splits = group_kfold.split(X_data, y_data, df_train2.group)
for train_index, test_index in splits:
    break
X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print(error, error_train)

perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

-1.045524140353588 -3.578226504262563


In [95]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
        j = 0
    else:
        j += 1
    
    if (len(columns_comb)>=70) or (j>=100):
        break
    

col dist_H_0_y 0 error 1.081751973702698 error_train 1.0550294977466406
col d_3_2 1 error 0.6005353595852118 error_train 0.49221504462929516
col dist_C_0_y 2 error 0.07807593070742562 error_train -0.19800769806785656
col atom_index_1_ aromatic 3 error 0.01719642781707902 error_train -0.3501746232959124
col atom_1_bond_lengths_mean 4 error -0.23493842088244496 error_train -0.7217087512759597
col bond_atom 5 error -0.2625240684316153 error_train -0.7541296419922382
col inv_dist1R 6 error -0.3680639812043885 error_train -0.8385121530493932
col d_3_1 7 error -0.3973806357142903 error_train -0.8972256047903716
col mulliken_atom_0 8 error -0.5487510827793718 error_train -1.1778260376962102
col dist_H_0_x 9 error -0.706022820177178 error_train -1.4175115229195456
col dist_O_0_y 10 error -0.7513907884756217 error_train -1.5692261567853103
col dist_C_1_x 11 error -0.8769879159143399 error_train -1.7417710394301336
col tertiary_angle_0 12 error -0.8897162953581453 error_train -1.8159052093935795

In [96]:
columns_comb

['dist_H_0_y',
 'd_3_2',
 'dist_C_0_y',
 'atom_index_1_ aromatic',
 'atom_1_bond_lengths_mean',
 'bond_atom',
 'inv_dist1R',
 'd_3_1',
 'mulliken_atom_0',
 'dist_H_0_x',
 'dist_O_0_y',
 'dist_C_1_x',
 'tertiary_angle_0',
 'dist_C_1_y',
 'vander_C.y',
 'dist_H_1_y',
 'mulliken_atom_1',
 'inv_dist0R',
 'd_1_0',
 'tertiary_distance_0',
 'tertiary_angle_2',
 'atom_index_1_explicit_valence',
 'dist_N_0_y',
 'inv_distPR',
 'dist_C_2_x',
 'vander_H.x',
 'd_4_2',
 'atom_index_0_eigv_max',
 'tertiary_distance_2',
 'dist_H_1_x',
 'dist_N_1_x',
 'dist_C_3_x',
 'cos_f0',
 'atom_index_1_sv_2',
 'max_molecule_atom_0_dist_xyz',
 'd_2_1']

In [8]:
for i in [4,5,6,7]:
    df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
    df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
    df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

    df_train2 = df_train2[df_train2['type']==i]
    df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
    df_train2 = df_train2.fillna(0)
    
    columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

    X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('init', i, error, error_train)

    perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
    df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
    df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
    df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
    df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
    
    columns_comb =[]
    best_error = 999
    his = []
    for col in df_feature_importances_i2.feature2.tolist():

        X_data = df_train2[[col]+columns_comb].values.astype('float32')
        y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

        group_kfold = GroupKFold(n_splits=8)
        splits = group_kfold.split(X_data, y_data, df_train2.group)
        for train_index, test_index in splits:
            break
        X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

        model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
        model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
        y_pred = model.predict(X_val)
        y_pred_train = model.predict(X_train)
        error = np.log(mean_absolute_error(y_val, y_pred))
        error_train = np.log(mean_absolute_error(y_train, y_pred_train))
        print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
        his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})

        if error < best_error:
            best_error = error
            columns_comb += [col]
            j = 0
        else:
            j += 1

        if (len(columns_comb)>=70) or (j>=100):
            print(columns_comb)
            break
    

init 4 -1.7209448613538423 -4.634338798915695
col d_3_1 0 error 0.5303010881908956 error_train 0.5265754225870725
col dist_H_1_x 1 error 0.09337929063177976 error_train 0.01434376211829868
col d_5_0 2 error -0.43390038861652086 error_train -0.7365402575581361
col d_4_0 3 error -0.8242287878098185 error_train -1.2240698395730305
col yukawa_H.x 4 error -0.8573470134371305 error_train -1.364166013819757
col inv_dist0 5 error -0.9685407472429141 error_train -1.615406298677814
col d_6_0 6 error -1.1764654098146645 error_train -1.8786303916849456
col d_4_1 7 error -1.3666404698858718 error_train -2.0905596893506067
col cos_c0 8 error -1.4672456431192764 error_train -2.263491546585578
col atom_3 9 error -1.4967113653752862 error_train -2.282500733554745
col dist_C_0_y 10 error -1.571925780803937 error_train -2.4222939691788716
col molecule_atom_index_0_dist_std_div 11 error -1.5839250503696614 error_train -2.451341627294883
col cos_c0_c1 12 error -1.6455990169470411 error_train -2.51819848725

col freqs_max 45 error -1.8923401411872978 error_train -3.086297416943557
col dist_1_neg_2 45 error -1.906752300647843 error_train -3.061041732650863
col link1 45 error -1.907779399374713 error_train -3.068989139482911
col dist_median_x 45 error -1.8883909342294614 error_train -3.069281438914519
col atom_index_1_eigv_gap 45 error -1.8998353138394355 error_train -3.070000110080792
col molecule_dist_mean 45 error -1.892068355390327 error_train -3.053611707966008
col inv_dist1E 45 error -1.892929140482324 error_train -3.0574379114996635
col d_9_1 45 error -1.9022377854322783 error_train -3.0732381102995174
col rc_B 45 error -1.9024863094449265 error_train -3.0781971709309035
col dist_neg_2 45 error -1.9170000170188397 error_train -3.0566875281289825
col dist_1_neg_3 45 error -1.9046938219634724 error_train -3.071024091213973
col distance_center0 45 error -1.8840323980388003 error_train -3.069334584395132
col molecule_dist_std_y 45 error -1.8785594055722077 error_train -3.0774323083739183


col dist_O_0_y 14 error -1.0218192810762572 error_train -1.2794645283514003
col d_4_1 15 error -1.0626507748042922 error_train -1.3292663433325151
col d_4_3 16 error -1.084118671892625 error_train -1.3569851612166113
col atom_index_1_cycle_size_mean 17 error -1.1146916884239872 error_train -1.384340897225559
col molecule_atom_index_0_dist_min_diff 18 error -1.1169767689189234 error_train -1.3863220416351771
col tertiary_atom_2 19 error -1.1248386656473688 error_train -1.3947635864182082
col atom_4 20 error -1.1277270001813773 error_train -1.3961299237412228
col cos_c0_f0 21 error -1.1342267337321712 error_train -1.4101071966706185
col tertiary_distance_3 22 error -1.1399375982860431 error_train -1.4168625689017214
col d_3_0 23 error -1.1412676386022822 error_train -1.4177359837513304
col dist_median_bond_y 24 error -1.1439317403664406 error_train -1.4206320005047208
col d_5_2 25 error -1.170177005649908 error_train -1.45050404404056
col adC3 26 error -1.1737048605468543 error_train -1.

col dist_C_0_y 25 error -1.7956827563447062 error_train -2.2964946652473706
col molecule_atom_index_0_dist_min_div 26 error -1.7932137310121992 error_train -2.290173418202293
col eem_1 26 error -1.7888884189494303 error_train -2.312193736585754
col d_5_2 26 error -1.8112681873725536 error_train -2.3184229511238956
col d_7_2 27 error -1.8236170546564876 error_train -2.3382976251622454
col vander_C.y 28 error -1.8197914447899641 error_train -2.3431832551499174
col dist_C_3_y 28 error -1.8313854605280568 error_train -2.352629550411763
col d_6_0 29 error -1.8428239001064641 error_train -2.366461748281432
col dihedral 30 error -1.8470773386213852 error_train -2.3759448747373773
col max_molecule_atom_0_dist_xyz 31 error -1.8625871401317682 error_train -2.4095588675305386
col d_7_3 32 error -1.8663632106557266 error_train -2.4206387656655255
col sd_molecule_atom_1_dist_xyz 33 error -1.8637401430769713 error_train -2.4191617989435126
col d_6_1 33 error -1.86971376599334 error_train -2.42582038

col d_4_3 26 error -2.1941575059159852 error_train -3.111734384744494
col cos_c0_f0 26 error -2.1923700892844193 error_train -3.1350822220513272
col atom_1_bond_lengths_std 26 error -2.197841177479741 error_train -3.130030411524301
col atom_index_1_n_cycle 27 error -2.200492502502627 error_train -3.1259231054101333
col d_4_2 28 error -2.203341796322101 error_train -3.1520688165150843
col cos_f0 29 error -2.2245253392950137 error_train -3.170133382407456
col d_5_2 30 error -2.2351560935025154 error_train -3.1928357103560328
col dist_C_2_y 31 error -2.2336199772893917 error_train -3.2157386537279646
col molecule_atom_index_1_dist_max_div 31 error -2.2338100591835124 error_train -3.2053044589691786
col max_molecule_atom_0_dist_xyz 31 error -2.2318503551291173 error_train -3.214519934819993
col dist_N_0_x 31 error -2.222244115511781 error_train -3.195927727504497
col dist_to_type_0_mean 31 error -2.2362380270173157 error_train -3.1902133243189157
col d_7_0 32 error -2.2336748438470737 erro

In [ ]:
init 0 -0.4324559369556779 -1.436310972165935
['tertiary_angle_0', 'inv_dist1R', 'd_4_3', 'yukawa_H.y', 'mulliken_atom_0', 'dist_to_type_mean', 'dist_O_0_x', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'dist_xyz', 'dist_C_0_y', 'd_3_2', 'atom_index_1_ hybridization', 'atom_index_1_cycle_size_mean', 'dist_O_0_y', 'eem_1', 'inv_distPE', 'd_4_2', 'inv_distPR', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_H_1_x', 'tertiary_distance_2', 'dist_C_3_x', 'dist_O_1_x', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'dist_C_2_x', 'mulliken_atom_1', 'cos_center1', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'dist_N_0_y', 'dist_C_1_y', 'inv_dist1E', 'distance_y', 'tertiary_angle_2', 'dist_N_0_x', 'd_2_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'adN1', 'd_4_0', 'dist_C_3_y', 'atom_3', 'distC0', 'tertiary_distance_4', 'tertiary_angle_5', 'd_5_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_y', 'dist_H_0_y', 'cos_f0', 'd_5_0', 'tertiary_distance_3', 'd_5_2', 'tertiary_atom_1', 'dist_C_4_x', 'cos_c0_f0', 'atom_index_0_sv_3', 'rc_C', 'cos_f1', 'tertiary_angle_8', 'dist_O_2_y', 'max_molecule_atom_1_dist_xyz', 'dist_F_0_y', 'atom_index_1_ aromatic', 'tertiary_angle_26', 'type_0']
init 1 -1.045524140353588 -3.578226504262563
['dist_H_0_y', 'd_3_2', 'dist_C_0_y', 'atom_index_1_ aromatic', 'atom_1_bond_lengths_mean', 'bond_atom', 'inv_dist1R', 'd_3_1', 'mulliken_atom_0', 'dist_H_0_x', 'dist_O_0_y', 'dist_C_1_x', 'tertiary_angle_0', 'dist_C_1_y', 'vander_C.y', 'dist_H_1_y', 'mulliken_atom_1', 'inv_dist0R', 'd_1_0', 'tertiary_distance_0', 'tertiary_angle_2', 'atom_index_1_explicit_valence', 'dist_N_0_y', 'inv_distPR', 'dist_C_2_x', 'vander_H.x', 'd_4_2', 'atom_index_0_eigv_max', 'tertiary_distance_2', 'dist_H_1_x', 'dist_N_1_x', 'dist_C_3_x', 'cos_f0', 'atom_index_1_sv_2', 'max_molecule_atom_0_dist_xyz', 'd_2_1']
init 2 -1.316571636180272 -2.044293659651798
['tertiary_atom_0', 'inv_dist0', 'dist_no_bond_min_x', 'atom_index_1_ hybridization', 'tertiary_angle_0', 'tertiary_angle_1', 'dist_O_0_x', 'cos_c0', 'd_5_2', 'tertiary_atom_1', 'cos_f0', 'dist_H_0_x', 'd_3_1', 'atom_index_1_degree', 'dist_C_0_y', 'adC2', 'dist_C_3_x', 'vander_O.y', 'mulliken_atom_1', 'atom_7', 'tertiary_angle_2', 'd_2_1', 'atom_3', 'd_5_1', 'd_6_2', 'd_4_1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_diff', 'd_4_2', 'dist_C_2_x', 'cos_c0_f0', 'd_6_0', 'dist_O_0_y', 'd_4_3', 'd_3_0', 'd_7_0', 'd_3_2', 'inv_dist0R', 'atom_8', 'dist_C_1_x', 'd_6_1', 'd_2_0', 'd_8_1', 'mulliken_atom_0', 'dist_N_0_x', 'atom_4', 'tertiary_distance_2', 'd_7_2', 'dist_C_0_x', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'bond_atom', 'd_7_1', 'd_4_0', 'distC0', 'atom_index_1_cycle_size_mean', 'cos_c0_c1', 'tertiary_angle_3', 'dist_O_1_x', 'atom_index_1_n_cycle', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max_div', 'atom_5', 'gap', 'cos_c1', 'dist_N_0_y', 'd_6_3', 'dist_C_3_y', 'inv_distP', 'dist_C_4_y']
init 3 -1.9251364757149136 -3.3697638772965974
['cos_c0', 'd_4_3', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_1', 'd_3_2', 'd_1_0', 'dist_H_0_y', 'mulliken_atom_0', 'mulliken_atom_1', 'dist_N_0_x', 'link0', 'tertiary_atom_2', 'dist_C_1_y', 'dist_C_1_x', 'cos_f0', 'dist_C_0_y', 'cos_f1', 'd_3_1', 'tertiary_distance_1', 'dist_O_0_y', 'cos_f0_f1', 'adC1', 'd_5_3', 'inv_distP', 'edge_4', 'd_6_2', 'dist_N_0_y', 'tertiary_distance_2', 'dist_O_0_x', 'cos_c1_f1', 'd_3_0', 'd_5_2', 'dist_C_0_x', 'adN1', 'cos_c0_f0', 'd_4_1', 'max_distance_y', 'dist_C_2_y', 'atom_5', 'adC3', 'dist_to_type_1_mean', 'vander_H.x', 'dist_C_3_y', 'dist_H_3_x', 'molecule_atom_index_0_dist_max_div', 'atom_7', 'dist_C_3_x', 'd_5_1', 'dist_H_3_y', 'atom_index_0_eigv_max', 'atom_6', 'dist_H_2_x', 'atom_index_1_sv_0', 'molecule_atom_index_1_dist_std_div', 'link1']
init 4 -1.7209448613563647 -4.63433879893567
['d_3_1', 'dist_H_1_x', 'd_5_0', 'd_4_0', 'yukawa_H.x', 'inv_dist0', 'd_6_0', 'd_4_1', 'cos_c0', 'atom_3', 'dist_C_0_y', 'molecule_atom_index_0_dist_std_div', 'cos_c0_c1', 'd_4_2', 'min_molecule_atom_0_dist_xyz', 'sd_molecule_atom_0_dist_xyz', 'd_2_1', 'adC2', 'd_3_0', 'dist_C_1_y', 'd_4_3', 'dist_H_0_x', 'vander_C.x', 'd_5_3', 'dist_H_1_y', 'tertiary_distance_3', 'd_2_0', 'dist_O_0_x', 'd_5_1', 'dist_O_0_y', 'adC3', 'inv_dist0R', 'dist_C_3_y', 'atom_index_1_ hybridization', 'cos_f0', 'dist_C_2_x', 'd_5_2', 'd_6_1', 'dist_C_0_x', 'atom_1_bond_lengths_min', 'mulliken_atom_1', 'distance_farthest_0', 'tertiary_distance_1', 'min_molecule_atom_1_dist_xyz', 'yukawa_O.y', 'atom_0_bond_lengths_max']
init 5 -1.236895026643203 -1.8522039705242035
['tertiary_angle_0', 'd_2_1', 'cos_c0', 'atom_1_bond_lengths_mean', 'd_3_1', 'd_2_0', 'tertiary_distance_1', 'd_3_2', 'tertiary_angle_1', 'cos_f0', 'tertiary_distance_2', 'dist_C_0_x', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'd_4_1', 'd_4_3', 'atom_index_1_cycle_size_mean', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_2', 'atom_4', 'cos_c0_f0', 'tertiary_distance_3', 'd_3_0', 'dist_median_bond_y', 'd_5_2', 'adC3', 'atom_5', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_div', 'gap', 'molecule_atom_index_1_dist_min_div', 'dist_O_0_x', 'cos_c1', 'dist_C_0_y', 'd_5_1', 'dist_N_0_y', 'dist_C_3_y', 'dist_no_bond_min_y', 'd_4_0', 'dist_N_0_x', 'd_4_2', 'max_molecule_atom_0_dist_xyz', 'cos_c0_c1', 'adC2', 'atom_index_1_n_cycle', 'd_5_0', 'd_6_1', 'dist_C_4_y', 'dist_O_1_y', 'd_7_2', 'tertiary_angle_2', 'd_6_2', 'mulliken_atom_1', 'atom_6', 'd_7_3', 'dist_O_1_x']
init 6 -1.822938904672602 -2.9384014064172073
['cos_c0_c1', 'atom_4', 'atom_5', 'molecule_atom_index_0_dist_min_diff', 'cos_c1', 'max_molecule_atom_1_dist_xyz', 'dist_to_type_std', 'd_3_2', 'cos_c0', 'dist_O_0_x', 'd_4_3', 'atom_6', 'dist_O_0_y', 'tertiary_atom_1', 'dist_C_2_y', 'd_4_2', 'dist_C_1_y', 'atom_7', 'tertiary_angle_1', 'dist_H_0_y', 'dist_no_bond_min_y', 'distance_c1', 'dist_C_2_x', 'linkM0', 'd_6_2', 'dist_C_0_y', 'd_5_2', 'd_7_2', 'dist_C_3_y', 'd_6_0', 'dihedral', 'max_molecule_atom_0_dist_xyz', 'd_7_3', 'd_6_1', 'dist_H_1_y', 'tertiary_atom_2', 'd_4_0', 'tertiary_atom_0', 'tertiary_angle_3', 'dist_C_0_x', 'dist_to_type_0_mean', 'dist_N_0_y', 'd_4_1', 'cos_c1_f1', 'cos_f0', 'dist_xyz', 'adC2', 'd_5_3', 'cos_f0_f1', 'gap', 'd_7_0', 'cos_f1', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_diff', 'd_2_1']
init 7 -2.059362626284956 -4.58353225619747
['cos_c0', 'tertiary_distance_1', 'cos_c1', 'd_3_2', 'tertiary_angle_1', 'tertiary_angle_0', 'atom_1_n_bonds', 'tertiary_distance_2', 'd_2_1', 'tertiary_angle_2', 'd_4_0', 'molecule_atom_index_0_dist_min_div', 'd_2_0', 'dist_H_0_x', 'd_3_1', 'cos_c0_c1', 'mulliken_atom_1', 'd_8_3', 'd_4_1', 'dist_C_0_y', 'd_3_0', 'atom_index_1_cycle_size_mean', 'dist_C_1_x', 'dist_C_2_x', 'adC2', 'adC1', 'atom_1_bond_lengths_std', 'atom_index_1_n_cycle', 'd_4_2', 'cos_f0', 'd_5_2', 'dist_to_type_0_mean', 'dist_O_0_x', 'molecule_atom_index_0_dist_std_diff', 'd_5_1', 'tertiary_angle_3', 'd_6_2', 'd_7_3']

In [13]:
base_param = {'columns': [], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 8}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {
        'learning_rate': 0.2,
    'num_leaves': 256,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0, 'n_estimators':1500, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}

In [14]:
type_columns = [['tertiary_angle_0', 'inv_dist1R', 'd_4_3', 'yukawa_H.y', 'mulliken_atom_0', 'dist_to_type_mean', 'dist_O_0_x', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'dist_xyz', 'dist_C_0_y', 'd_3_2', 'atom_index_1_ hybridization', 'atom_index_1_cycle_size_mean', 'dist_O_0_y', 'eem_1', 'inv_distPE', 'd_4_2', 'inv_distPR', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_H_1_x', 'tertiary_distance_2', 'dist_C_3_x', 'dist_O_1_x', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'dist_C_2_x', 'mulliken_atom_1', 'cos_center1', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'dist_N_0_y', 'dist_C_1_y', 'inv_dist1E', 'distance_y', 'tertiary_angle_2', 'dist_N_0_x', 'd_2_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'adN1', 'd_4_0', 'dist_C_3_y', 'atom_3', 'distC0', 'tertiary_distance_4', 'tertiary_angle_5', 'd_5_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_y', 'dist_H_0_y', 'cos_f0', 'd_5_0', 'tertiary_distance_3', 'd_5_2', 'tertiary_atom_1', 'dist_C_4_x', 'cos_c0_f0', 'atom_index_0_sv_3', 'rc_C', 'cos_f1', 'tertiary_angle_8', 'dist_O_2_y', 'max_molecule_atom_1_dist_xyz', 'dist_F_0_y', 'atom_index_1_ aromatic', 'tertiary_angle_26', 'type_0'],
['dist_H_0_y', 'd_3_2', 'dist_C_0_y', 'atom_index_1_ aromatic', 'atom_1_bond_lengths_mean', 'bond_atom', 'inv_dist1R', 'd_3_1', 'mulliken_atom_0', 'dist_H_0_x', 'dist_O_0_y', 'dist_C_1_x', 'tertiary_angle_0', 'dist_C_1_y', 'vander_C.y', 'dist_H_1_y', 'mulliken_atom_1', 'inv_dist0R', 'd_1_0', 'tertiary_distance_0', 'tertiary_angle_2', 'atom_index_1_explicit_valence', 'dist_N_0_y', 'inv_distPR', 'dist_C_2_x', 'vander_H.x', 'd_4_2', 'atom_index_0_eigv_max', 'tertiary_distance_2', 'dist_H_1_x', 'dist_N_1_x', 'dist_C_3_x', 'cos_f0', 'atom_index_1_sv_2', 'max_molecule_atom_0_dist_xyz', 'd_2_1'],
['tertiary_atom_0', 'inv_dist0', 'dist_no_bond_min_x', 'atom_index_1_ hybridization', 'tertiary_angle_0', 'tertiary_angle_1', 'dist_O_0_x', 'cos_c0', 'd_5_2', 'tertiary_atom_1', 'cos_f0', 'dist_H_0_x', 'd_3_1', 'atom_index_1_degree', 'dist_C_0_y', 'adC2', 'dist_C_3_x', 'vander_O.y', 'mulliken_atom_1', 'atom_7', 'tertiary_angle_2', 'd_2_1', 'atom_3', 'd_5_1', 'd_6_2', 'd_4_1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_diff', 'd_4_2', 'dist_C_2_x', 'cos_c0_f0', 'd_6_0', 'dist_O_0_y', 'd_4_3', 'd_3_0', 'd_7_0', 'd_3_2', 'inv_dist0R', 'atom_8', 'dist_C_1_x', 'd_6_1', 'd_2_0', 'd_8_1', 'mulliken_atom_0', 'dist_N_0_x', 'atom_4', 'tertiary_distance_2', 'd_7_2', 'dist_C_0_x', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'bond_atom', 'd_7_1', 'd_4_0', 'distC0', 'atom_index_1_cycle_size_mean', 'cos_c0_c1', 'tertiary_angle_3', 'dist_O_1_x', 'atom_index_1_n_cycle', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max_div', 'atom_5', 'gap', 'cos_c1', 'dist_N_0_y', 'd_6_3', 'dist_C_3_y', 'inv_distP', 'dist_C_4_y'],
['cos_c0', 'd_4_3', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_1', 'd_3_2', 'd_1_0', 'dist_H_0_y', 'mulliken_atom_0', 'mulliken_atom_1', 'dist_N_0_x', 'link0', 'tertiary_atom_2', 'dist_C_1_y', 'dist_C_1_x', 'cos_f0', 'dist_C_0_y', 'cos_f1', 'd_3_1', 'tertiary_distance_1', 'dist_O_0_y', 'cos_f0_f1', 'adC1', 'd_5_3', 'inv_distP', 'edge_4', 'd_6_2', 'dist_N_0_y', 'tertiary_distance_2', 'dist_O_0_x', 'cos_c1_f1', 'd_3_0', 'd_5_2', 'dist_C_0_x', 'adN1', 'cos_c0_f0', 'd_4_1', 'max_distance_y', 'dist_C_2_y', 'atom_5', 'adC3', 'dist_to_type_1_mean', 'vander_H.x', 'dist_C_3_y', 'dist_H_3_x', 'molecule_atom_index_0_dist_max_div', 'atom_7', 'dist_C_3_x', 'd_5_1', 'dist_H_3_y', 'atom_index_0_eigv_max', 'atom_6', 'dist_H_2_x', 'atom_index_1_sv_0', 'molecule_atom_index_1_dist_std_div', 'link1'],
['d_3_1', 'dist_H_1_x', 'd_5_0', 'd_4_0', 'yukawa_H.x', 'inv_dist0', 'd_6_0', 'd_4_1', 'cos_c0', 'atom_3', 'dist_C_0_y', 'molecule_atom_index_0_dist_std_div', 'cos_c0_c1', 'd_4_2', 'min_molecule_atom_0_dist_xyz', 'sd_molecule_atom_0_dist_xyz', 'd_2_1', 'adC2', 'd_3_0', 'dist_C_1_y', 'd_4_3', 'dist_H_0_x', 'vander_C.x', 'd_5_3', 'dist_H_1_y', 'tertiary_distance_3', 'd_2_0', 'dist_O_0_x', 'd_5_1', 'dist_O_0_y', 'adC3', 'inv_dist0R', 'dist_C_3_y', 'atom_index_1_ hybridization', 'cos_f0', 'dist_C_2_x', 'd_5_2', 'd_6_1', 'dist_C_0_x', 'atom_1_bond_lengths_min', 'mulliken_atom_1', 'distance_farthest_0', 'tertiary_distance_1', 'min_molecule_atom_1_dist_xyz', 'yukawa_O.y', 'atom_0_bond_lengths_max'],
['tertiary_angle_0', 'd_2_1', 'cos_c0', 'atom_1_bond_lengths_mean', 'd_3_1', 'd_2_0', 'tertiary_distance_1', 'd_3_2', 'tertiary_angle_1', 'cos_f0', 'tertiary_distance_2', 'dist_C_0_x', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'd_4_1', 'd_4_3', 'atom_index_1_cycle_size_mean', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_2', 'atom_4', 'cos_c0_f0', 'tertiary_distance_3', 'd_3_0', 'dist_median_bond_y', 'd_5_2', 'adC3', 'atom_5', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_div', 'gap', 'molecule_atom_index_1_dist_min_div', 'dist_O_0_x', 'cos_c1', 'dist_C_0_y', 'd_5_1', 'dist_N_0_y', 'dist_C_3_y', 'dist_no_bond_min_y', 'd_4_0', 'dist_N_0_x', 'd_4_2', 'max_molecule_atom_0_dist_xyz', 'cos_c0_c1', 'adC2', 'atom_index_1_n_cycle', 'd_5_0', 'd_6_1', 'dist_C_4_y', 'dist_O_1_y', 'd_7_2', 'tertiary_angle_2', 'd_6_2', 'mulliken_atom_1', 'atom_6', 'd_7_3', 'dist_O_1_x'],
['cos_c0_c1', 'atom_4', 'atom_5', 'molecule_atom_index_0_dist_min_diff', 'cos_c1', 'max_molecule_atom_1_dist_xyz', 'dist_to_type_std', 'd_3_2', 'cos_c0', 'dist_O_0_x', 'd_4_3', 'atom_6', 'dist_O_0_y', 'tertiary_atom_1', 'dist_C_2_y', 'd_4_2', 'dist_C_1_y', 'atom_7', 'tertiary_angle_1', 'dist_H_0_y', 'dist_no_bond_min_y', 'distance_c1', 'dist_C_2_x', 'linkM0', 'd_6_2', 'dist_C_0_y', 'd_5_2', 'd_7_2', 'dist_C_3_y', 'd_6_0', 'dihedral', 'max_molecule_atom_0_dist_xyz', 'd_7_3', 'd_6_1', 'dist_H_1_y', 'tertiary_atom_2', 'd_4_0', 'tertiary_atom_0', 'tertiary_angle_3', 'dist_C_0_x', 'dist_to_type_0_mean', 'dist_N_0_y', 'd_4_1', 'cos_c1_f1', 'cos_f0', 'dist_xyz', 'adC2', 'd_5_3', 'cos_f0_f1', 'gap', 'd_7_0', 'cos_f1', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_diff', 'd_2_1'],
['cos_c0', 'tertiary_distance_1', 'cos_c1', 'd_3_2', 'tertiary_angle_1', 'tertiary_angle_0', 'atom_1_n_bonds', 'tertiary_distance_2', 'd_2_1', 'tertiary_angle_2', 'd_4_0', 'molecule_atom_index_0_dist_min_div', 'd_2_0', 'dist_H_0_x', 'd_3_1', 'cos_c0_c1', 'mulliken_atom_1', 'd_8_3', 'd_4_1', 'dist_C_0_y', 'd_3_0', 'atom_index_1_cycle_size_mean', 'dist_C_1_x', 'dist_C_2_x', 'adC2', 'adC1', 'atom_1_bond_lengths_std', 'atom_index_1_n_cycle', 'd_4_2', 'cos_f0', 'd_5_2', 'dist_to_type_0_mean', 'dist_O_0_x', 'molecule_atom_index_0_dist_std_diff', 'd_5_1', 'tertiary_angle_3', 'd_6_2', 'd_7_3']]

In [ ]:
df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
df_train2_plus = df_train2_plus.rename(columns={'id':'index'})
df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
df_train2 = df_train2.fillna(0)

In [ ]:
df_test2 = pd.read_pickle(f'{file_folder}/df_test2.gzde', compression='gzip')
df_test2_plus = pd.read_pickle(f'{file_folder}/df_test2_plus.gzde', compression='gzip')
df_test2_plus = df_test2_plus.rename(columns={'id':'index'})
df_test2 = pd.merge(df_test2, df_test2_plus, how='left', on='index')
df_test2 = df_test2.fillna(0)

In [ ]:
df_train = df_train2
df_test = df_test2

In [ ]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    
    param = base_param.copy()
    param['columns'] = type_columns[t]
    
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), param, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)
    

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [ ]:
for i in [0,1,2,3, 4,5,6,7]:
    df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
    df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
    df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

    df_train2 = df_train2[df_train2['type']==i]
    df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
    df_train2 = df_train2.fillna(0)
    
    columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

    X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

        model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
        model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
        y_pred = model.predict(X_val)
        y_pred_train = model.predict(X_train)
        error = np.log(mean_absolute_error(y_val, y_pred))
        error_train = np.log(mean_absolute_error(y_train, y_pred_train))
        print('init', i, error, error_train)

    

In [11]:
for i in [4,5]:
    df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
    df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
    df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

    df_train2 = df_train2[df_train2['type']==i]
    df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
    df_train2 = df_train2.fillna(0)
    
    columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

    X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
    y_data = df_train2['fc'].values.astype('float32')

    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('init', i, error, error_train)

    perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
    df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
    df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
    df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
    df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
    
    columns_comb =[]
    best_error = 999
    his = []
    for col in df_feature_importances_i2.feature2.tolist():

        X_data = df_train2[[col]+columns_comb].values.astype('float32')
        y_data = df_train2['fc'].values.astype('float32')

        group_kfold = GroupKFold(n_splits=8)
        splits = group_kfold.split(X_data, y_data, df_train2.group)
        for train_index, test_index in splits:
            break
        X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

        model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
        model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
        y_pred = model.predict(X_val)
        y_pred_train = model.predict(X_train)
        error = np.log(mean_absolute_error(y_val, y_pred))
        error_train = np.log(mean_absolute_error(y_train, y_pred_train))
        print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
        his.append({'i':i, 'col':[col]+columns_comb, 'error':error, 'error_train':error_train})

        if error < best_error:
            best_error = error
            columns_comb += [col]
            j = 0
        else:
            j += 1

        if (len(columns_comb)>=70) or (j>=100):
            print(columns_comb)
            break
    

init 4 -1.73425395539969 -4.662130830818556
col d_2_1 0 error 0.6919646224905218 error_train 0.6861049770366239
col sd_molecule_atom_0_dist_xyz 1 error 0.3477969369094592 error_train 0.2694142929520069
col d_3_1 2 error -0.2877401646036626 error_train -0.5105203823079685
col molecule_atom_index_1_dist_min_div 3 error -0.6025518696527677 error_train -1.0327917710881815
col inv_dist0 4 error -0.8961313892287021 error_train -1.4577853843966941
col d_6_0 5 error -1.1299736922605181 error_train -1.7717212373983522
col d_4_0 6 error -1.2953754971768623 error_train -1.994161910417355
col dist_H_0_x 7 error -1.379238696334995 error_train -2.1023202410919994
col d_5_0 8 error -1.4762208379878325 error_train -2.209274466252702
col dist_C_0_y 9 error -1.5632897414030158 error_train -2.3197278268171715
col d_4_1 10 error -1.6144293585491418 error_train -2.3786200526313914
col adC2 11 error -1.6403037678154302 error_train -2.4271607363900953
col dist_H_1_x 12 error -1.6340908732168804 error_train -

col vander_C.y 41 error -1.8992976534695154 error_train -3.0254591231912333
col atom_index_1_dist_q.05 41 error -1.8986959481932344 error_train -3.0031101355414753
col atom_index_1_dist_q.01 41 error -1.9089108245591158 error_train -2.9997112975389597
col dist_N_0_x 41 error -1.903261613172774 error_train -3.0166994209352818
col atom_6 41 error -1.9128792715139518 error_train -3.013819313029904
col dist_no_bond_min_y 41 error -1.9056605330708518 error_train -3.0092334788631594
col distance_y 41 error -1.9098074062028103 error_train -3.0055448900179944
col linkM0 41 error -1.893490461202152 error_train -3.0357315634848403
col dist_C_4_y 41 error -1.9042983021491273 error_train -3.021392022198676
col atom_index_1_degree 41 error -1.903527742063095 error_train -3.011215695156435
col inv_dist0E 41 error -1.9099108064556092 error_train -3.0140046381459644
col tertiary_angle_4 41 error -1.9000714803525425 error_train -3.01723834478111
col d_7_3 41 error -1.9156679542147395 error_train -3.022

col linkM1 44 error -1.9124841103158787 error_train -3.06273002966575
col atom_index_1_sv_4 44 error -1.9119859245482325 error_train -3.128359255382627
col sd_molecule_atom_1_dist_xyz 44 error -1.910970769126465 error_train -3.095087093150892
col molecule_dist_max 44 error -1.9024918643672342 error_train -3.0916586568503193
col atom_8 44 error -1.911785347081515 error_train -3.068317224676495
col rc_C 44 error -1.9024726452282708 error_train -3.0898817315406752
col atom_index_1_dist_q.99 44 error -1.888690963457935 error_train -3.116724034858022
col atom_index_0_dist_q.01 44 error -1.9111597704885543 error_train -3.0880132923281587
col molecule_type_dist_mean 44 error -1.8916595526292923 error_train -3.0665727322916636
col tertiary_distance_10 44 error -1.911608850932381 error_train -3.1026967389858804
col dist_O_1_x 44 error -1.9120164288686674 error_train -3.0696114008958064
col molecule_atom_index_1_dist_max_div 44 error -1.9181661092751165 error_train -3.0997190580995455
col link1 

col tertiary_angle_10 45 error -1.9078973165558424 error_train -3.132169491710612
col atom_index_1_dist_q.75 45 error -1.8962201971726274 error_train -3.12818145243096
col alpha 45 error -1.9007422353922807 error_train -3.125055899478734
col atom_index_0_t_dist_factors_0 45 error -1.9063028019347499 error_train -3.1183076644880945
col molecule_couples 45 error -1.9012457410548547 error_train -3.11750004849945
col atom_index_0_t_dist_core 45 error -1.8893389318145617 error_train -3.135276888403106
col x_dist 45 error -1.8965254374595606 error_train -3.142697997664134
col dx 45 error -1.8925243176360593 error_train -3.147780558669751
col dist_mean_no_bond_x 45 error -1.9027259097525093 error_train -3.1332226993362653
col tertiary_atom_11 45 error -1.9234993699915799 error_train -3.0951369839487586
col range_dist_no_bond_x 45 error -1.9087575409929325 error_train -3.1473256355570594
col atom_index_1_p_dist_factors 45 error -1.8978590943043163 error_train -3.130828838959158
col coulomb_C.y

col eem_0 53 error -1.3377243559649532 error_train -1.7171388581527331
col adC1 53 error -1.3401923554242725 error_train -1.7149563608567593
col molecule_type_0_dist_std_diff 53 error -1.3361765977601618 error_train -1.7121167923225216
col d_7_0 53 error -1.3405632278137578 error_train -1.7206090115483377
col atom_1_bond_lengths_min 53 error -1.3380797787645948 error_train -1.7129034353574244
col molecule_atom_index_0_dist_std_diff 53 error -1.3332809607592508 error_train -1.7149998171823133
col d_8_3 53 error -1.3405898287531968 error_train -1.726809606846561
col d_7_3 53 error -1.3433559852770565 error_train -1.7216635459827803
col inv_dist0 53 error -1.335311195375755 error_train -1.7146881248325372
col atom_6 53 error -1.3415843849226334 error_train -1.7182344820015827
col dist_O_1_x 53 error -1.3438260996667717 error_train -1.721534735805599
col dist_H_1_y 53 error -1.3402242874716854 error_train -1.7211794838744379
col d_9_2 53 error -1.3380412592683117 error_train -1.72026219141

['tertiary_angle_0', 'd_2_1', 'cos_c0', 'd_3_1', 'atom_1_bond_lengths_mean', 'd_2_0', 'tertiary_distance_1', 'd_3_2', 'tertiary_angle_1', 'dist_C_0_x', 'tertiary_distance_2', 'cos_f0', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'dist_O_0_y', 'd_4_3', 'd_4_1', 'atom_index_1_cycle_size_mean', 'dist_H_1_x', 'tertiary_distance_3', 'atom_4', 'molecule_atom_index_1_dist_min_div', 'cos_c0_f0', 'adC3', 'dist_O_0_x', 'd_5_2', 'dist_C_3_y', 'dist_N_0_x', 'gap', 'dist_no_bond_min_y', 'tertiary_atom_1', 'adC2', 'd_5_1', 'max_molecule_atom_0_dist_xyz', 'd_4_2', 'cos_c1', 'dist_N_0_y', 'atom_2', 'cos_c0_c1', 'atom_index_1_n_cycle', 'mulliken_atom_1', 'd_6_2', 'd_7_2', 'd_5_0', 'dist_C_0_y', 'd_6_1', 'd_5_3', 'dist_O_1_y', 'd_6_0', 'd_6_3', 'tertiary_angle_3', 'd_7_1', 'dist_xyz']


In [ ]:
init 0 -0.4240007342041508 -1.431623147336867
['tertiary_angle_0', 'd_4_3', 'mulliken_atom_0', 'inv_dist1R', 'dist_O_0_x', 'dist_to_type_mean', 'yukawa_H.y', 'dist_to_type_1_mean', 'atom_1_n_bonds', 'atom_1_bond_lengths_mean', 'dist_C_0_y', 'atom_index_1_cycle_size_mean', 'inv_distPE', 'eem_1', 'dist_O_0_y', 'd_3_2', 'inv_dist0R', 'd_4_2', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_1_x', 'dist_H_2_x', 'dist_O_1_x', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'dist_C_2_x', 'dist_C_3_x', 'dist_O_1_y', 'dist_H_0_y', 'dist_C_1_y', 'dist_N_0_y', 'cos_center1', 'dist_H_2_y', 'd_4_0', 'inv_dist1E', 'tertiary_angle_3', 'mulliken_atom_1', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'adC1', 'tertiary_angle_2', 'tertiary_angle_4', 'dist_H_1_y', 'tertiary_angle_1', 'dist_C_3_y', 'adN1', 'd_5_3', 'tertiary_distance_3', 'cos_f0', 'tertiary_distance_4', 'dist_C_4_y', 'distC1', 'tertiary_angle_6', 'dist_H_3_x', 'dist_N_1_y', 'dist_C_4_x', 'd_5_0', 'd_5_2', 'molecule_atom_index_0_dist_max_diff', 'dist_N_1_x', 'd_6_3', 'd_8_1', 'molecule_type_dist_std_diff']
init 1 -1.0572756952991065 -3.6080878667781926
['dist_H_0_y', 'dist_C_0_y', 'd_3_2', 'atom_index_1_ aromatic', 'mulliken_atom_0', 'atom_1_bond_lengths_mean', 'd_3_1', 'bond_atom', 'inv_dist1R', 'dist_H_0_x', 'dist_O_0_y', 'dist_C_1_y', 'tertiary_angle_0', 'dist_C_1_x', 'dist_H_1_y', 'tertiary_angle_1', 'tertiary_angle_2', 'vander_H.x', 'mulliken_atom_1', 'd_2_0', 'dist_N_1_x', 'dist_C_2_x', 'atom_index_1_explicit_valence', 'dist_H_1_x', 'cos_center0', 'molecule_atom_index_0_dist_max_div', 'linkM1', 'adC1', 'cos_f1', 'dist_1_pos_2', 'molecule_atom_index_0_dist_std_div', 'd_5_1', 'yukawa_O.x', 'd_7_0', 'sd_molecule_atom_1_dist_xyz', 'coulomb_N.x', 'molecule_atom_index_1_dist_max_diff', 'min_molecule_atom_1_dist_xyz', 'tertiary_angle_7']
init 2 -1.3100157805662835 -2.053662149159018
['tertiary_atom_0', 'inv_dist0R', 'dist_no_bond_min_x', 'atom_index_1_ hybridization', 'tertiary_angle_0', 'molecule_atom_index_1_dist_min_diff', 'd_3_1', 'dist_O_0_x', 'tertiary_atom_1', 'cos_f0', 'molecule_atom_index_0_dist_min_diff', 'dist_C_2_x', 'tertiary_angle_1', 'd_5_2', 'mulliken_atom_1', 'atom_index_1_cycle_size_mean', 'tertiary_atom_2', 'adC2', 'dist_C_3_x', 'd_6_2', 'd_4_1', 'dist_H_0_x', 'tertiary_distance_1', 'd_2_1', 'dist_O_0_y', 'tertiary_angle_2', 'atom_7', 'd_6_1', 'cos_c0', 'cos_c0_f0', 'd_3_2', 'd_4_2', 'atom_3', 'd_5_1', 'atom_4', 'dist_C_0_y', 'd_7_0', 'd_6_0', 'd_4_3', 'dist_N_0_x', 'd_7_2', 'atom_1_bond_lengths_mean', 'dist_C_1_x', 'mulliken_atom_0', 'd_8_1', 'adC1', 'atom_5', 'distC0', 'atom_8', 'd_7_1', 'dist_C_0_x', 'dist_C_1_y', 'bond_atom', 'd_5_0', 'adN1', 'tertiary_angle_3', 'molecule_atom_index_0_dist_max_div', 'gap', 'd_4_0', 'tertiary_distance_3', 'dist_N_0_y', 'dist_O_1_x', 'd_5_3', 'd_8_2', 'd_9_1', 'eem_1', 'molecule_atom_index_0_dist_max_diff', 'd_1_0', 'tertiary_atom_5', 'dist_O_2_x']
init 3 -1.9096400429221767 -3.341257864919127
['cos_c0', 'd_4_3', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_1', 'distance_y', 'mulliken_atom_1', 'dist_H_0_y', 'd_3_2', 'mulliken_atom_0', 'dist_N_0_x', 'link0', 'atom_3', 'tertiary_atom_2', 'dist_C_1_x', 'dist_C_1_y', 'cos_f1', 'cos_f0', 'dist_O_0_x', 'dist_C_0_y', 'molecule_atom_index_0_dist_min_div', 'lumo', 'd_5_3', 'd_3_1', 'cos_f0_f1', 'd_6_2', 'd_3_0', 'tertiary_distance_1', 'dist_H_1_x', 'cos_c0_f0', 'd_4_1', 'cos_c1_f1', 'yukawa_H.x', 'd_4_2', 'cos_c1', 'd_5_2', 'distance_c1', 'tertiary_angle_2', 'inv_distP', 'dist_N_0_y', 'dist_O_0_y', 'adN1', 'dist_C_2_y', 'gap', 'inv_distPE', 'adC3', 'dist_C_2_x', 'max_distance_y', 'd_7_2', 'dist_C_3_y', 'dist_H_2_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_diff', 'tertiary_distance_8']
init 4 -1.7342539554053475 -4.6621308307859435
['d_2_1', 'sd_molecule_atom_0_dist_xyz', 'd_3_1', 'molecule_atom_index_1_dist_min_div', 'inv_dist0', 'd_6_0', 'd_4_0', 'dist_H_0_x', 'd_5_0', 'dist_C_0_y', 'd_4_1', 'adC2', 'cos_c0_c1', 'd_4_3', 'atom_3', 'd_4_2', 'd_5_3', 'dist_C_0_x', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'cos_c0', 'd_5_1', 'dist_C_2_x', 'd_7_0', 'molecule_atom_index_0_dist_min_diff', 'mulliken_atom_0', 'dist_N_1_x', 'tertiary_distance_1', 'cos_f0', 'dist_O_0_y', 'atom_1_n_bonds', 'd_6_1', 'atom_4', 'd_6_3', 'tertiary_distance_4', 'd_2_0', 'mulliken_atom_1', 'dist_to_type_std', 'dist_O_0_x', 'molecule_atom_index_1_dist_min_diff', 'tertiary_angle_1', 'd_7_3', 'atom_index_1_n_cycle', 'dist_H_4_y', 'molecule_atom_index_1_dist_mean_div']
init 5 -1.242733358958433 -1.8567441567469962
['tertiary_angle_0', 'd_2_1', 'cos_c0', 'd_3_1', 'atom_1_bond_lengths_mean', 'd_2_0', 'tertiary_distance_1', 'd_3_2', 'tertiary_angle_1', 'dist_C_0_x', 'tertiary_distance_2', 'cos_f0', 'dist_H_0_x', 'molecule_atom_index_0_dist_min_diff', 'dist_O_0_y', 'd_4_1', 'd_4_3', 'atom_index_1_cycle_size_mean', 'dist_H_1_x', 'tertiary_distance_3', 'atom_4', 'molecule_atom_index_1_dist_min_div', 'cos_c0_f0', 'dist_O_0_x', 'adC3', 'd_5_2', 'dist_C_3_y', 'd_3_0', 'dist_C_2_x', 'dist_N_0_x', 'gap', 'dist_no_bond_min_y', 'adC2', 'tertiary_atom_1', 'd_5_1', 'd_4_2', 'max_molecule_atom_0_dist_xyz', 'cos_c1', 'dist_N_0_y', 'atom_5', 'cos_c0_c1', 'atom_index_1_n_cycle', 'mulliken_atom_1', 'd_6_2', 'd_7_2', 'd_5_0', 'd_5_3', 'dist_C_0_y', 'd_6_1', 'tertiary_angle_2', 'dist_O_1_y', 'd_1_0', 'd_6_3', 'distC0', 'molecule_atom_index_0_dist_max_diff', 'd_7_1', 'dist_O_1_x', 'd_9_2', 'd_8_1']
init 6 -1.8318285333947395 -2.9502284158868815
['cos_c0_c1', 'atom_4', 'max_molecule_atom_1_dist_xyz', 'molecule_atom_index_0_dist_min_diff', 'atom_5', 'd_3_2', 'dist_O_0_x', 'dist_to_type_std', 'atom_6', 'cos_c1', 'tertiary_atom_1', 'dist_O_0_y', 'd_4_3', 'dist_C_2_y', 'molecule_atom_index_0_dist_min_div', 'dist_C_1_y', 'atom_7', 'tertiary_angle_1', 'cos_c0', 'dist_H_0_y', 'dist_C_0_y', 'dist_C_2_x', 'd_4_2', 'dist_no_bond_min_y', 'd_6_2', 'd_5_2', 'adC3', 'dist_H_0_x', 'd_6_0', 'd_7_2', 'd_7_3', 'molecule_atom_index_0_dist_max_diff', 'd_6_1', 'linkM0', 'd_6_3', 'atom_8', 'dist_C_1_x', 'd_5_1', 'cos_f0', 'dist_N_0_x', 'd_5_3', 'd_8_3', 'max_distance_y', 'dist_N_0_y', 'inv_distPR', 'is_cis', 'tertiary_atom_2', 'cos_f1', 'd_8_2', 'dist_C_3_x', 'dist_O_1_x', 'molecule_atom_index_1_dist_mean_diff']
init 7 -2.075667043774986 -4.591618371332592
['cos_c0', 'cos_c1', 'd_3_2', 'tertiary_distance_1', 'tertiary_angle_1', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'd_4_0', 'atom_1_n_bonds', 'd_2_1', 'd_2_0', 'd_3_1', 'atom_index_1_degree', 'dist_H_0_x', 'tertiary_angle_2', 'molecule_atom_index_0_dist_mean_diff', 'd_8_3', 'cos_c0_c1', 'd_4_1', 'dist_C_0_y', 'mulliken_atom_1', 'dist_C_1_x', 'molecule_atom_index_0_dist_std_diff', 'd_4_2', 'atom_index_1_cycle_size_mean', 'atom_index_0_eigv_gap', 'atom_index_1_n_cycle', 'dist_C_2_x', 'd_5_2', 'cos_f0', 'd_3_0', 'd_4_3', 'dist_to_type_0_mean', 'dist_C_3_y', 'atom_1_bond_lengths_std', 'd_5_1', 'dist_O_0_x', 'dist_O_0_y', 'dist_C_2_y', 'dist_H_0_y', 'd_6_2', 'd_7_2', 'min_molecule_atom_1_dist_xyz', 'inv_dist1R', 'dist_H_2_y', 'd_9_1', 'dist_C_3_x']